In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# modules we'll use
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# read in all our data
cardiodf = pd.read_csv("/kaggle/input/cardiogoodfitness/CardioGoodFitness.csv")

In [ ]:
#first check out of the data
cardiodf.head(5)

# How many missing data points do we have?

What percentage of the values in the dataset are missing?

In [ ]:
# how many total missing values do we have?
#totale values in our dataset
total_cells = np.product(cardiodf.shape)
#description of  missing values in each column
missing_values_count = cardiodf.isnull().sum()
print(missing_values_count.sort_values(ascending=False))
#totale missing values in our dataset
total_missing = missing_values_count.sum()

# percent of data that is missing
percent_missing = (total_missing/total_cells) * 100
print('% of data that is missing :',percent_missing)

The Data set looking good so far , it has `0% missing values` , so the next step is find relationships between features to maybe find hidden insghits.

# Data Description

In [ ]:
cardiodf.describe(include='all')

# First thoughts
***Average Age of users ?***

arround 29 Years Old (mean =~28.79 )

***min Age of users ?***

18 Years Old (min = 18.0)

***max Age of users ?***

50 Years Old (max= 50.0)






# Answering a data-driven question

***what is the most dominating Gender using CardioGoodFitness Products ( specifing the protduct and the marital statues) ?***

From the description above , it appears that `Male` users using `Product TM195` that are most likely to be in a `Relationship aka Partnered` are the most dominating users in the dataSet !

# 2) Viz & interpetation


***Counting Gender per Product***

In [ ]:
sns.countplot(x='Product', hue = 'Gender', data = cardiodf)


***Age Density***

In [ ]:
sns.distplot(cardiodf['Age'])

***Dominating age per gender***

In [ ]:
cardiodf.hist(by='Gender',column = 'Age')

In [ ]:
sns.boxenplot(x='Gender',y='Age',data=cardiodf)

***How do income and age affect the custumer's choice of products?***
(Inspired by **Vahideh Dashti [Kaggle Master](https://www.kaggle.com/vahidehdashti)**)

In [ ]:
sns.scatterplot(x='Age', y='Income',data=cardiodf, hue = 'Product')
plt.show()

***How does Education affect user's Income?***

In [ ]:
plt.figure();
sns.pointplot(data=cardiodf, x='Education', y='Income');

as Expected to be :)

***Any relationship between products and Education lvls ?***

In [ ]:
#create boxplot of education per product
fig = plt.figure();
sns.boxplot(data=cardiodf, x='Product', y='Education');
plt.xlabel('Product');
plt.ylabel('Education');
plt.title('Education per Product', fontsize=12);

it appears that ***more educated costumers*** tend to buy the ***TM789*** 

***relationship between products and fitness-lvls ?***

In [ ]:
#create boxplot of education per product
fig = plt.figure();
sns.boxplot(data=cardiodf, x='Product', y='Fitness');
plt.xlabel('Product');
plt.ylabel('Fitness');
plt.title('Education per Product', fontsize=12);

the TM798 is most likely to be used by users with heigh lvl fitness rates!

***Best Sold Product ?***
(Inspired by **JordanRich [Data Scientist and Business Analytics Professional](https://www.kaggle.com/jordanrich/cardiogoodfitness-analysis-of-demographics#Context:)**)

In [ ]:
#create counts of data for plotting Product categorical variables
products = np.unique(cardiodf['Product'], return_counts=True)
print('products = {}\n'.format(products))


In [ ]:
#function to display percent alongside count
def spec(x):
    return '{:.1f}%\n({:.0f})'.format(x, np.sum(products[1])*x/100);

#create pie chart of products sold
fig = plt.figure(figsize=[5,5]);
patches, texts, autotexts = plt.pie(list(products[1]),labels=list(products[0]), autopct=spec, shadow=True, startangle=90);

plt.title('Total Products Sold', fontsize=16, fontweight='bold');

***Marital Statues and Product purshase?***

In [ ]:
#create counts of data for plotting MaritalStatus categorical variables
maritals = np.unique(cardiodf['MaritalStatus'], return_counts=True)
print('MaritalStatus = {}\n'.format(maritals))

In [ ]:
#create barplot of MaritalStatus and products sold
fig = plt.figure();
sns.barplot(x=list(maritals[0]), y=list(maritals[1]));
plt.xlabel('maritals',  fontweight='bold');
plt.ylabel('Counts',  fontweight='bold');
plt.title('effects of Maritual Statues on products purshased');

as expected to be ! ***partened users*** are ***more likely*** to be motivated ***to buy fitness products*** then Single users !

# Preparing Data for Prediction


Now let's start by encoding  `MaritalStatus` and `gender` to get ride of the `Object-type` values in our DataSet and to prepare it for prediction, but before that let's find out if all of our values in both `MaritalStatus` and `gender` `columns` have only two unique values


In [ ]:
uniques_MaritalStatus = cardiodf['MaritalStatus'].unique()
uniques_Gender = cardiodf['Gender'].unique()
uniques_Product = cardiodf['Product'].unique()
uniques_MaritalStatus.sort()
uniques_Gender.sort()
uniques_Product.sort()
print('unique entries in MaritalStatus Column :',uniques_MaritalStatus)
print('unique entries in Gender Column :',uniques_Gender)
print('unique entries in Product Column :',uniques_Product)

It looks like we dont have inconsistent data entries too , which is greate!.

So let's Ordinal Encoding the 3 columns!.

As a reminder : `Ordinal encoding` assigns each unique value to a different integer, as an example, in the `Gender Column` the ordinal Encoding will assign `1 -> Male and 2 -> Female`.
Same thing with `MaritalStatus & Product`.

Note: The values won't be the one that you provided, since internally the `fit` method use `numpy.unique` which gives result sorted in `alphabetic order` and NOT BY order of appearance.(Special thanks to **abcdaire [answer](https://stackoverflow.com/questions/56502864/using-ordinalencoder-to-transform-categorical-values) in stackoverflow**)

So... instead of using OrdinalEncoder , I'm going to use `map` function to transform all the object values to `integer` values ( i will also leave the OrdinalEncoder code version in a comment if you are interrested in ) simply because we dont have much values to transfer.!



In [ ]:
# All categorical columns
object_cols = [col for col in cardiodf.columns if cardiodf[col].dtype == "object"]

# Columns that can be safely ordinal encoded
good_label_cols = [col for col in object_cols if 
                   set(cardiodf[col]).issubset(set(cardiodf[col]))]
print('Categorical columns that will be ordinal encoded:', good_label_cols)

Let's Start the Action!

**OrdinalEncoder_Version**

In [ ]:
'''from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()

# Apply ordinal encoder 
cardiodf[good_label_cols] = ordinal_encoder.fit_transform(cardiodf[good_label_cols])
cardiodf.head(5)'''

***Map_Version***

In [ ]:
product_dict={'TM195': 1,'TM498': 2,'TM798':3}
genre_dict={'Female':2, 'Male':1}
marital_dict={'Partnered':2, 'Single':1}

cardiodf['Product'] = cardiodf.Product.map(product_dict)
cardiodf['Gender'] = cardiodf.Gender.map(genre_dict)
cardiodf['MaritalStatus'] = cardiodf.MaritalStatus.map(marital_dict)
cardiodf.head(5)




# Feature Engineering


***MI Scores, why MI and not Correlation?***

`Mutual Information (aka MI score)` : it measures a relationship between two quantities. 
The advantage of mutual information is that `it can detect any kind of relationship`, 
while `correlation only detects linear relationships.`


The (MI) between 2 quantities is a measure of `one quantity reduces uncertainty about the other.`

When MI is 0 : the quantities are independent ( ex: a feature and a target have 0 MI ==> dot use that feature in prediction)

`Please Note:` A feature is only useful when its relationship with the target is one your model can learn. 
Just because a feature has a high MI score doesn't mean your model will be able to do anything 
You may need to transform the feature first to expose the association.  
Also ..., Feature must be continous feature :  
Float dtype is not discrete. ==> convert it  
Categoricals (object or categorial dtype) ==> label encoding

In [ ]:
from sklearn.feature_selection import mutual_info_regression

# Utility functions from Tutorial (Feature Engineering @Kaggle_learn)
def make_mi_scores(X, y):
    X = X.copy()
    for colname in X.select_dtypes(["object", "category"]):
        X[colname], _ = X[colname].factorize()
    # All discrete features should now have integer dtypes
    discrete_features = [pd.api.types.is_integer_dtype(t) for t in X.dtypes]
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features, random_state=0)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores


def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")

In [ ]:
# Set Matplotlib defaults
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True)
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=14,
    titlepad=10,
)

features = ['Fitness','Usage']
sns.relplot(
    x="value", y="Product", col="variable", data=cardiodf.melt(id_vars="Product", value_vars=features), facet_kws=dict(sharex=False),
);

In [ ]:
X = cardiodf.copy()
y = X.pop('Miles')

mi_scores = make_mi_scores(X, y)

print(mi_scores.head(8))
# print(mi_scores.tail(8))  # uncomment to see bottom 20

plt.figure(dpi=100, figsize=(8, 5))
plot_mi_scores(mi_scores.head(8))
# plot_mi_scores(mi_scores.tail(8))  # uncomment to see bottom 20

Do the scores seem reasonable? Do the high scoring features represent things i'd think most people would value in fitness Product?  
well... the Fitness and Usage seems pretty obvious for me, because i think if someone is focusing on doing his daily steps the `Fitness` feature should have high MI score.  
Also, the `Product` if someone has more money, he should be pretty confortable investing it in his health on better equipement no?  
But i also expected the Income lvl and education lvls should have more MI scores, this is why i m going to investigate them more in the next step!

In [ ]:
sns.catplot(x="Fitness", y="Miles", data=cardiodf, kind="boxen");

As expected, Pepole with high fitness lvls tends to walk more miles  

In [ ]:
sns.catplot(x="Usage", y="Miles", data=cardiodf, kind="boxen");

Excatly the same as the Fitness lvls!, the more you use the product the more you walk more :)

# Prediction

which type of ML should we use ?: Supervised Learning (A simple Linear reg)    
why?  
Because we use regression when there is a continuous range of outputs. ex:  House size (input), output - price  
Also..., Classification - output is this or that (well defined and discrete)

***Selecting The Prediction Target***

In [ ]:
y = cardiodf['Miles']

***Choosing "Features"***

In [ ]:
# list of features below
X= cardiodf[['Fitness','Usage']]

**Building Our Model**

The steps to building and using a model are:  
**Define**: What type of model will it be? A decision tree? Some other type of model? Some other parameters of the model type are specified too.  
**Fit**: Capture patterns from provided data. This is the heart of modeling.  
**Predict**: Just what it sounds like  
**Evaluate**: Determine how accurate the model's predictions are.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import mean_squared_error
from math import sqrt
from statistics import mean
#define
model = LinearRegression()
#Split ( default recommended testing data size is 20% of the totale Data)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
#Fit 
model.fit(X_train,y_train)
#Predict
print('equation mean coeff =',mean(model.coef_))
print('equation Intercept =',model.intercept_)
print('finale miles predicted equation:')
print('Miles =',"{:.2f}".format(mean(model.coef_)),'x Fitness + ',"{:.2f}".format(mean(model.coef_)),'x Usage + ',"{:.2f}".format(model.intercept_))
